In [7]:
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import time
import sys, io, os

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [9]:
import json
from tqdm import tqdm

In [10]:
from torchvision import datasets, transforms

In [11]:
device = torch.device('cuda:0')

## Data Loader

In [13]:
cifar_train = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

cifar_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

train_dataset = datasets.CIFAR10(root="../../_Datasets/cifar10/", train=True, download=True, transform=cifar_train)
test_dataset = datasets.CIFAR10(root="../../_Datasets/cifar10/", train=False, download=True, transform=cifar_test)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
# cifar_train = transforms.Compose([
#     transforms.RandomCrop(size=32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.5071, 0.4865, 0.4409],
#         std=[0.2009, 0.1984, 0.2023],
#     ),
# ])

# cifar_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.5071, 0.4865, 0.4409],
#         std=[0.2009, 0.1984, 0.2023],
#     ),
# ])

# train_dataset = datasets.CIFAR100(root="../../_Datasets/cifar100/", train=True, download=False, transform=cifar_train)
# test_dataset = datasets.CIFAR100(root="../../_Datasets/cifar100/", train=False, download=False, transform=cifar_test)

In [15]:
batch_size = 32 ## 32-c10, 128-c100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, 
                                           shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, 
                                          shuffle=False, num_workers=4)

In [16]:
len(train_dataset.classes)

10

## Model

In [17]:
import cifar_resnet_custom as resnet

In [26]:
inplanes=23
# model = resnet.cifar_resnet20(inplanes=inplanes)
model = resnet.cifar2_resnet20(inplanes=inplanes, num_classes=len(train_dataset.classes))

In [27]:
n_params = np.sum([p.numel() for p in model.parameters()])
print(f"Number of parameters is: {n_params}") ## 1148152

Number of parameters is: 1446917


In [28]:
model(torch.randn(1, 3, 32, 32))

tensor([[ 0.7915, -0.5020, -0.0469, -0.5315,  0.0015, -0.0276,  0.3534, -0.5536,
         -0.3480, -0.4690]], grad_fn=<AddmmBackward0>)

In [29]:
model.to(device)

CifarResNet2(
  (conv1): Sequential(
    (0): Conv2d(3, 23, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): Dropout2d(p=0.1, inplace=False)
  )
  (bn1): BatchNorm2d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(23, 23, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): Dropout2d(p=0.1, inplace=False)
      )
      (bn1): BatchNorm2d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Sequential(
        (0): Conv2d(23, 23, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): Dropout2d(p=0.1, inplace=False)
      )
      (bn2): BatchNorm2d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(23, 23, kernel_size=(3, 3), stride

In [65]:
asfsdfadf

NameError: name 'asfsdfadf' is not defined

## Optimizer and Loss

In [30]:
EPOCHS = 200

In [31]:
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1,
#                       momentum=0.9, weight_decay=5e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [32]:
# model_name = f"resnet20_inplanes_{inplanes}_params_{n_params}.pth"
model_name = f"resnet20_b4_inplanes_{inplanes}_params_{n_params}.pth" ## b4 uses cifar2_resnet
# model_name = f"resnet20_b4_c100_inplanes_{inplanes}_params_{n_params}.pth"
# model_name = f"resnet20_b4_c100_inplanes_{inplanes}_params_{n_params}_bs128.pth"

In [33]:
train_accs = []
test_accs = []
train_losses, test_losses = [], []

## Training Code

In [34]:
## Following is copied from 
### https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

# Training
def train(epoch):
#     print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print(f"[Train] {epoch} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    train_accs.append(100.*correct/total)
    train_losses.append(train_loss/(batch_idx+1))
    return

In [35]:
best_acc = -1
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    test_losses.append(test_loss/(batch_idx+1))
    
    # Save checkpoint.
    acc = 100.*correct/total
    test_accs.append(acc)
    if acc > best_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/{model_name}')
        best_acc = acc

In [36]:
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
resume = False

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load(f'./models/{model_name}')
    model.load_state_dict(checkpoint['model'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

In [37]:
def save_accs():
    with open(f"./models/{model_name}_accs_loss.json", 'w') as f:
        d = {
            "train_accs":train_accs,
            "test_accs":test_accs,
            "train_loss":train_losses,
            "test_loss":test_losses,
        }
        json.dump(d, f, indent=0)

In [38]:
### Train the whole damn thing

for epoch in range(start_epoch, start_epoch+EPOCHS): ## for 200 epochs
    train(epoch)
    test(epoch)
    scheduler.step()
    save_accs()

100%|█████████████████████████████████████████████████| 1563/1563 [00:23<00:00, 67.47it/s]


[Train] 0 Loss: 1.754 | Acc: 34.542 17271/50000


100%|██████████████████████████████████████████████████| 313/313 [00:01<00:00, 232.19it/s]


[Test] 0 Loss: 1.407 | Acc: 47.490 4749/10000
Saving..


100%|█████████████████████████████████████████████████| 1563/1563 [00:22<00:00, 68.04it/s]


...

[Test] 198 Loss: 0.287 | Acc: 92.960 9296/10000


100%|█████████████████████████████████████████████████| 1563/1563 [00:23<00:00, 67.37it/s]


[Train] 199 Loss: 0.086 | Acc: 96.934 48467/50000


100%|██████████████████████████████████████████████████| 313/313 [00:01<00:00, 233.00it/s]

[Test] 199 Loss: 0.285 | Acc: 93.030 9303/10000


In [ ]:
adsrsrs

In [39]:
best_acc, model_name

(93.14, 'resnet20_b4_inplanes_23_params_1446917.pth')

### Load Best Model State

In [16]:
print('==> Resuming from checkpoint..')
assert os.path.isdir('models'), 'Error: no checkpoint directory found!'
checkpoint = torch.load(f'./models/{model_name}')
model.load_state_dict(checkpoint['model'])
best_acc = checkpoint['acc']
start_epoch = checkpoint['epoch']

==> Resuming from checkpoint..


In [17]:
### Linear -> (92.96, 184)
### Distance -> (92.64, 199)
best_acc, start_epoch

(87.87, 159)

## Experiment Logs

In [ ]:
####### CIFAR 10
### Resnet20 , wo Dropout, inplanes=11, --> (89.21, 'resnet_20_inplanes_11_params_129359.pth')
### Resnet20 , w Dropout, inplanes=11, --> (89.64, 'resnet20_inplanes_11_params_129359.pth')

### Resnet20, w Dropout, inplanes=33, 3 blocks --> (94.74, 'resnet20_inplanes_33_params_1153261.pth')

### Resnet20, w Dropout, inplanes=21, 4 blocks --> (93.2, 'resnet20_b4_inplanes_21_params_1206691.pth')
### Resnet20, w Dropout, inplanes=9, 4 blocks --> (87.87, 'resnet20_b4_inplanes_9_params_222967.pth')
### Resnet20, w Dropout, inplanes=13, 4 blocks --> (90.51, 'resnet20_b4_inplanes_13_params_463707.pth')
### Resnet20, w Dropout, inplanes=17, 4 blocks --> (92.25, 'resnet20_b4_inplanes_17_params_791615.pth')
### Resnet20, w Dropout, inplanes=23, 4 blocks --> (93.14, 'resnet20_b4_inplanes_23_params_1446917.pth')


In [ ]:
######## Cifar 100 --> default: 128 batch size

### (62.44, 'resnet20_b4_c100_inplanes_12_params_404080_bs128.pth') 
### (63.7, 'resnet20_b4_c100_inplanes_12_params_404080.pth') --> 32 batch size
### (65.43, 'resnet20_b4_c100_inplanes_15_params_627655_bs128.pth')
### (67.01, 'resnet20_b4_c100_inplanes_17_params_803945_bs128.pth')
### (57.18, 'resnet20_b4_c100_inplanes_9_params_229537_bs128.pth')